In [90]:
#Read an Excel  file and extract 3 values,

import pandas as pd

# Path to the Excel file
file_path = r"C:\Users\Randall\Downloads\Listado de Movimiento de Bancos.xls"

# Read the Excel file
data = pd.read_excel(file_path)

# Extract the values from the specified columns
columns_to_extract = ['Recibo', 'Concepto', 'Débito']
extracted_data = data[columns_to_extract].copy()

# Filter out rows where 'Recibo' is longer than 10 characters
extracted_data = extracted_data[extracted_data['Recibo'].str.len() <= 10]

# Split 'Concepto' at the first space into 'code' and 'name'
def split_concepto(value):
    if isinstance(value, str):
        parts = value.split(' ', 1)
        if len(parts) == 2:
            return pd.Series({'code': parts[0], 'name': parts[1]})
        else:
            return pd.Series({'code': parts[0], 'name': ''})
    else:
        return pd.Series({'code': '', 'name': ''})

extracted_data[['code', 'name']] = extracted_data['Concepto'].apply(split_concepto)

# Create 'flag' column: 1 if 'Recibo' and 'code' do not match, else 0
extracted_data['flag'] = (extracted_data['Recibo'].astype(str) != extracted_data['code'].astype(str)).astype(int)

# Create 'final' column: if 'Recibo' and 'code' are equal, use 'Recibo'; else use the longest one
extracted_data['final'] = extracted_data.apply(
    lambda row: row['Recibo'] if str(row['Recibo']) == str(row['code']) else max([str(row['Recibo']), str(row['code'])], key=len),
    axis=1
)

extracted_data['final']

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


0     HMHCTBQDJ3
1     HM9F8N92TN
2     HMDXW9PN54
3     HMACKQQ9RA
4     HMFSKRMFRD
         ...    
66    HMFZMPW5RF
67    HMFQ224SCX
68    HMYXNBE4RW
69    HMYCTD22WH
70    HMM4BQKTAH
Name: final, Length: 69, dtype: object

In [91]:
import pandas as pd

# Path to the Excel file
file_path = r"C:\Users\Randall\Downloads\reservations.csv"

# Read the Excel file
data = pd.read_csv(file_path)
data['Ingresos'] = data['Ingresos'][1::]

Reservaciones = data[data['Estado'] == 'Huésped anterior']

# Display the first few rows of the dataframe

In [92]:
Reservaciones

,Código de confirmación,Estado,Nombre del huésped,Contacta,Número de adultos,Número de niños,Número de bebés,Fecha de inicio,Hasta,Número de noches,Reservado,Anuncio,Ingresos
1,HMD3JTFPK9,Huésped anterior,Liseth Astudillo,+593 98 146 1964,4,1,0,14/9/2025,15/9/2025,1,2025-09-14,Aurora | Vista Increíble | Depto Lujo | Factura,$142.44
3,HMMZDW4YDD,Huésped anterior,Lorena Cuellar,NaN,3,0,0,12/9/2025,13/9/2025,1,2025-09-12,Aurora | Vista Increíble | Depto Lujo | Factura,$110.85
4,HMPCAFCWC2,Huésped anterior,Victoria Mora Martinez,NaN,3,1,0,6/9/2025,9/9/2025,3,2025-09-02,Luxury Apartment Stunning View Casablanca Invoice,$359.40
5,HMYXNBE4RW,Huésped anterior,Yolanda Viera,NaN,5,0,0,29/8/2025,31/8/2025,2,2025-08-28,Luxury Apartment Stunning View Casablanca Invoice,$251.29
7,HMDKPHMKEH,Huésped anterior,Daniel Velasquez,NaN,2,0,0,27/8/2025,28/8/2025,1,2025-08-26,Luxury Suite | Casa Blanca Alta | Pool | Invoice,$92.65
8,HMYCTD22WH,Huésped anterior,Andy Montalvan Riera,NaN,5,0,0,24/8/2025,27/8/2025,3,2025-08-21,Luxury Apartment Stunning View Casablanca Invoice,$376.94
10,HM4PCKWSE9,Huésped anterior,José Jaramillo Aguilar,NaN,4,1,0,23/8/2025,24/8/2025,1,2025-08-10,Luxury Apartment Stunning View Casablanca Invoice,$125.65
11,HMM4BQKTAH,Huésped anterior,Jony Bastidas,NaN,2,1,0,22/8/2025,24/8/2025,2,2025-08-22,Luxury Suite | Casa Blanca Alta | Pool | Invoice,$224.40
12,HMD4FQ3CDF,Huésped anterior,Gabriel Osorio,NaN,2,2,0,21/8/2025,23/8/2025,2,2025-08-16,Luxury Apartment Stunning View Casablanca Invoice,$230.36
13,HMEFFK5TE8,Huésped anterior,Diana Toledo,NaN,4,1,0,19/8/2025,21/8/2025,2,2025-08-12,Luxury Apartment Stunning View Casablanca Invoice,$209.42


In [93]:
# Convert 'Ingresos' column in Reservaciones to numeric, removing any non-numeric characters
Reservaciones['Ingresos'] = Reservaciones['Ingresos'].replace('[\$,]', '', regex=True).astype(float)

# Step 1: Check for matches by 'Código de confirmación' and 'final'
matches = Reservaciones.merge(extracted_data, left_on='Código de confirmación', right_on='final', how='inner')

# Step 2: Filter rows where 'Ingresos' matches 'Débito' for confirmed matches
confirmed_matches = matches[matches['Ingresos'] == matches['Débito']]

# Step 3: Identify rows in Reservaciones that do not match by 'Código de confirmación'
non_matches = Reservaciones[~Reservaciones['Código de confirmación'].isin(extracted_data['final'])]

# Step 4: Compare by price for non-matching rows
price_matches = non_matches.merge(extracted_data, left_on='Ingresos', right_on='Débito', how='inner')

# Step 5: Extract 'Código de confirmación' and 'final' for price matches
price_matches_extracted = price_matches[['Código de confirmación', 'final']]

# Display results


print("\nExtracted 'Código de confirmación' and 'final' for Price Matches:")
price_matches_extracted


Extracted 'Código de confirmación' and 'final' for Price Matches:


<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
C:\Users\Randall\AppData\Local\Temp\ipykernel_53724\38366815.py:2: SyntaxWarning: invalid escape sequence '\$'
  Reservaciones['Ingresos'] = Reservaciones['Ingresos'].replace('[\$,]', '', regex=True).astype(float)
C:\Users\Randall\AppData\Local\Temp\ipykernel_53724\38366815.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Reservaciones['Ingresos'] = Reservaciones['Ingresos'].replace('[\$,]', '', regex=True).astype(float)


,Código de confirmación,final
0,HM4PCKWSE9,HMDT8FJCFB
1,HMEFFK5TE8,HMKKF8WNNX
2,HMEFFK5TE8,HMRQPABFHC
3,HMEFFK5TE8,HMFZMPW5RF
4,HMBEEDNE5M,HMKKF8WNNX
5,HMBEEDNE5M,HMRQPABFHC
6,HMBEEDNE5M,HMFZMPW5RF


In [94]:
# Get 'Código de confirmación' values from Reservaciones that are not in 'final' column of extracted_data
non_matching_reservaciones_data = Reservaciones.loc[~Reservaciones['Código de confirmación'].isin(extracted_data['final'])]
print(non_matching_reservaciones_data.shape)
non_matching_reservaciones_data


(11, 13)


,Código de confirmación,Estado,Nombre del huésped,Contacta,Número de adultos,Número de niños,Número de bebés,Fecha de inicio,Hasta,Número de noches,Reservado,Anuncio,Ingresos
1,HMD3JTFPK9,Huésped anterior,Liseth Astudillo,+593 98 146 1964,4,1,0,14/9/2025,15/9/2025,1,2025-09-14,Aurora | Vista Increíble | Depto Lujo | Factura,142.44
3,HMMZDW4YDD,Huésped anterior,Lorena Cuellar,NaN,3,0,0,12/9/2025,13/9/2025,1,2025-09-12,Aurora | Vista Increíble | Depto Lujo | Factura,110.85
4,HMPCAFCWC2,Huésped anterior,Victoria Mora Martinez,NaN,3,1,0,6/9/2025,9/9/2025,3,2025-09-02,Luxury Apartment Stunning View Casablanca Invoice,359.40
7,HMDKPHMKEH,Huésped anterior,Daniel Velasquez,NaN,2,0,0,27/8/2025,28/8/2025,1,2025-08-26,Luxury Suite | Casa Blanca Alta | Pool | Invoice,92.65
10,HM4PCKWSE9,Huésped anterior,José Jaramillo Aguilar,NaN,4,1,0,23/8/2025,24/8/2025,1,2025-08-10,Luxury Apartment Stunning View Casablanca Invoice,125.65
12,HMD4FQ3CDF,Huésped anterior,Gabriel Osorio,NaN,2,2,0,21/8/2025,23/8/2025,2,2025-08-16,Luxury Apartment Stunning View Casablanca Invoice,230.36
13,HMEFFK5TE8,Huésped anterior,Diana Toledo,NaN,4,1,0,19/8/2025,21/8/2025,2,2025-08-12,Luxury Apartment Stunning View Casablanca Invoice,209.42
15,HMBEEDNE5M,Huésped anterior,Tammy Bolaños,NaN,3,0,0,17/8/2025,19/8/2025,2,2025-08-15,Luxury Apartment Stunning View Casablanca Invoice,209.42
16,HMCCRCFHH3,Huésped anterior,Veronica Jeaneth Ona Serrano,NaN,3,0,0,16/8/2025,17/8/2025,1,2025-08-13,Luxury Suite | Casa Blanca Alta | Pool | Invoice,98.00
17,HM4J52A9KF,Huésped anterior,Frank Gonzalez,NaN,4,0,0,15/8/2025,16/8/2025,1,2025-08-15,Aurora | Vista Increíble | Depto Lujo | Factura,97.92
